In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import locale
import mibian
import numpy as np

In [10]:
# URL de la página a hacer scraping
url = 'https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35'

# Realizar la petición HTTP GET a la página
response = requests.get(url)

In [11]:
def obtener_dataframe(response, tipo_tabla):
    """
    Realiza el web scraping y devuelve un dataframe con los datos obtenidos,
    dependiendo si el tipo de tabla es 'opciones' o 'futuros'.
    
    Args:
    - response: La respuesta HTTP obtenida.
    - tipo_tabla: Tipo de la tabla a buscar ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe con los datos de la tabla.
    """
    # Determinar el ID de la tabla y si se necesita manejar el atributo data-tipo
    if tipo_tabla == 'opciones':
        id_tabla = 'tblOpciones'
        es_opcion = True
    elif tipo_tabla == 'futuros':
        id_tabla = 'Contenido_Contenido_tblFuturos'
        es_opcion = False
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', id=id_tabla)
        all_rows_data = []
        
        if table:
            rows = table.find_all('tr', class_='text-right')
            
            for row in rows:
                cells = row.find_all('td')
                row_data = [cell.text.strip() for cell in cells]
                
                if es_opcion:
                    data_tipo = row.get('data-tipo', 'No especificado')
                    row_data.insert(0, data_tipo)
                    
                all_rows_data.append(row_data)
                
            return pd.DataFrame(all_rows_data)
        else:
            print('No se encontró la tabla con el id especificado.')
            return pd.DataFrame()
    else:
        print('Error al realizar la petición HTTP:', response.status_code)
        return pd.DataFrame()

In [12]:
def tratar_dataframe(df, tipo_tabla):
    """
    Transforma el dataframe según si es de opciones o de futuros.
    
    Args:
    - df: Dataframe a transformar.
    - tipo_tabla: Tipo de la tabla ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe transformado.
    """
    if tipo_tabla == 'opciones':
        # Especificar los nombres de columna para opciones
        df.columns = ['Class', 'Strike', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        df['Tipo'] = df['Class'].str[:3]
        df['Fecha'] = pd.to_datetime(df['Class'].str[3:], format='%Y%m%d').dt.strftime('%d-%m-%Y')
        df = df.drop(['Class'], axis=1)
        
        # Transformaciones adicionales para opciones
        df['Strike'] = df['Strike'].str.replace('.', '').str.replace(',', '.').astype(float)
        df['Ant'] = pd.to_numeric(df['Ant'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d-%m-%Y')
        
        # Seleccionando solo las columnas deseadas para opciones
        df = df.loc[:, ['Tipo', 'Fecha', 'Strike', 'Ant']]
        
    elif tipo_tabla == 'futuros':
        # Especificar los nombres de columna para futuros
        df.columns = ['Vencimiento', 'Tipo', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        
        # Configurar locale a español
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8' if locale.windows_locale is None else 'Spanish')
        df['Vencimiento'] = pd.to_datetime(df['Vencimiento'], format='%d %b %Y')
        df['Ant'] = pd.to_numeric(df['Ant'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        
        # Seleccionando solo las columnas deseadas para futuros
        df = df.loc[:, ['Vencimiento', 'Ant']]
        
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    return df

In [84]:
# Opciones
df_opciones = obtener_dataframe(response, 'opciones')
df_opciones = tratar_dataframe(df_opciones, 'opciones')

In [85]:
# Futuros
df_futuros = obtener_dataframe(response, 'futuros')
df_futuros = tratar_dataframe(df_futuros, 'futuros')

In [86]:
# Filtrar para obtener solo las opciones de compra (calls) y de venta (puts)
df_calls = df_opciones[df_opciones['Tipo'] == 'OCE']
df_puts = df_opciones[df_opciones['Tipo'] == 'OPE']

# Crear un diccionario donde cada fecha es una clave y el valor es el DataFrame correspondiente
calls_por_fecha = {fecha: grupo for fecha, grupo in df_calls.groupby('Fecha')}
puts_por_fecha = {fecha: grupo for fecha, grupo in df_puts.groupby('Fecha')}

# Incluir estos diccionarios bajo las claves 'Call' y 'Put' en el diccionario 'resultados'
resultados = {
    'Call': calls_por_fecha,
    'Put': puts_por_fecha
}


In [87]:
df_futuros

,Vencimiento,Ant
0,2024-04-19,10644.0
1,2024-05-17,10610.0
2,2024-06-21,10617.0


In [88]:
price_sub = df_futuros.loc[0, 'Ant']
rfr = 0
dia_futuro = df_futuros.loc[0, 'Vencimiento']

In [89]:
price_sub

10644.0

In [90]:
df_calls

,Tipo,Fecha,Strike,Ant
1,OCE,2024-04-19,8400.0,2244.0
2,OCE,2024-04-19,8500.0,2144.0
5,OCE,2024-04-19,8600.0,2044.0
6,OCE,2024-04-19,8700.0,1944.0
9,OCE,2024-04-19,8800.0,1844.0
...,...,...,...,...
1599,OCE,2025-06-20,13300.0,38.0
1600,OCE,2025-06-20,13400.0,32.0
1603,OCE,2025-06-20,13500.0,27.0
1604,OCE,2025-06-20,13600.0,23.0


In [91]:
fechas = [dia for dia in resultados['Call'].keys()]

In [92]:
def volatilidad(precio_f, vencimiento_f, precio_strike, interest_rate, vencimiento_c, precio_c):
    """
    Calcula la volatilidad implícita de una opción call utilizando el modelo de Black-Scholes.
    
    Parámetros:
    precio_f (float): Precio actual del activo subyacente.
    vencimiento_f (int): Fecha de vencimiento del activo subyacente.
    precio_strike (float): Precio de ejercicio de la opción call.
    interest_rate (float): Tasa de interés libre de riesgo anual.
    vencimiento_c (int): Fecha de vencimiento de la opción call.
    precio_c (float): Precio de la opción call.

    Retorna:
    float: La volatilidad implícita de la opción call, o None si no es calculable.
    """

    variacion_dias = vencimiento_c - vencimiento_f
    if variacion_dias != 0:
        call = mibian.BS([precio_f, precio_strike, interest_rate, variacion_dias], callPrice=precio_c)
        volatilidad_implicita = call.impliedVolatility
    else:
        volatilidad_implicita = None 

    return volatilidad_implicita


In [93]:
df_calls

,Tipo,Fecha,Strike,Ant
1,OCE,2024-04-19,8400.0,2244.0
2,OCE,2024-04-19,8500.0,2144.0
5,OCE,2024-04-19,8600.0,2044.0
6,OCE,2024-04-19,8700.0,1944.0
9,OCE,2024-04-19,8800.0,1844.0
...,...,...,...,...
1599,OCE,2025-06-20,13300.0,38.0
1600,OCE,2025-06-20,13400.0,32.0
1603,OCE,2025-06-20,13500.0,27.0
1604,OCE,2025-06-20,13600.0,23.0


In [117]:
def volatilidad(row, precio_f, vencimiento_f, interest_rate):
    """
    Función para calcular la volatilidad implícita de una opción call.
    
    Parámetros:
    row (pd.Series): Fila del DataFrame que representa una opción.
    precio_f (float): Precio actual del activo subyacente obtenido de df_futuros.
    vencimiento_f (datetime.date or pd.Timestamp): Fecha de vencimiento del futuro obtenida de df_futuros.
    interest_rate (float): Tasa de interés libre de riesgo.

    Retorna:
    float: La volatilidad implícita de la opción call, o None si no es calculable.
    """
    if row.isna().any():
        return None
    else:
        precio_strike = row['Strike']
        vencimiento_c = row['Fecha']
        precio_c = row['Ant']

        # Asegúrate de que vencimiento_f y vencimiento_c son objetos de fecha
        if isinstance(vencimiento_f, pd.Timestamp):
            vencimiento_f = vencimiento_f.to_pydatetime().date()
        if isinstance(vencimiento_c, pd.Timestamp):
            vencimiento_c = vencimiento_c.to_pydatetime().date()

        # Calcula la diferencia de días como un número entero
        variacion_dias = (vencimiento_c - vencimiento_f).days
        if variacion_dias != 0:
            call = mibian.BS([precio_f, precio_strike, interest_rate, variacion_dias], callPrice=precio_c)
            return call.impliedVolatility
        else:
            return None

In [118]:
df_2 = df_calls.iloc[300:500]
df_2

,Tipo,Fecha,Strike,Ant
600,OCE,2024-05-17,9500.0,1116.0
603,OCE,2024-05-17,9550.0,1067.0
604,OCE,2024-05-17,9600.0,1019.0
607,OCE,2024-05-17,9650.0,971.0
608,OCE,2024-05-17,9700.0,923.0
...,...,...,...,...
991,OCE,2024-09-20,9200.0,1504.0
992,OCE,2024-09-20,9300.0,1416.0
995,OCE,2024-09-20,9400.0,1330.0
996,OCE,2024-09-20,9500.0,1244.0


In [112]:
# Obtiene los valores del DataFrame df_futuros
price_sub = df_futuros.loc[0, 'Ant']
rfr = 0
dia_futuro = df_futuros.loc[0, 'Vencimiento']

strike = 9600.0
precio_opcion = 1160.0


fecha_vencimiento = pd.Timestamp('2024-09-20 00:00:00')
variacion = (fecha_vencimiento - dia_futuro).days

In [114]:
variacion

154

In [121]:
mibian.BS([price_sub, strike, rfr, variacion], callPrice=precio_opcion).impliedVolatility

17.63916015625

In [ ]:
# Calcula la volatilidad implícita directamente
df_2['Volatilidad Implícita'] = volatilidad(df_2, price_sub, dia_futuro, rfr)

In [122]:
vol_list = []
# Itera sobre cada fila en df_2 usando iterrows()
for index, row in df_2.iterrows():
    # Calcula la volatilidad implícita usando la función volatilidad
    # Asumiendo que la función 'volatilidad' está definida y puede manejar los argumentos correctamente
    implied_vol = volatilidad(row, price_sub, dia_futuro, rfr)
    # Asigna el valor calculado a la columna 'Volatilidad Implícita' en el índice correspondiente
    vol_list.append(implied_vol)
    print(index)

600
603
604
607
608


KeyboardInterrupt: 

In [41]:
df_2

,Tipo,Fecha,Strike,Ant,Volatilidad Implícita
1,OCE,2024-04-19,8400.0,2244.0,None
2,OCE,2024-04-19,8500.0,2144.0,None
5,OCE,2024-04-19,8600.0,2044.0,None
6,OCE,2024-04-19,8700.0,1944.0,None
9,OCE,2024-04-19,8800.0,1844.0,None
10,OCE,2024-04-19,8900.0,1744.0,None
13,OCE,2024-04-19,9000.0,1644.0,None
14,OCE,2024-04-19,9100.0,1544.0,None
17,OCE,2024-04-19,9200.0,1444.0,None
18,OCE,2024-04-19,9250.0,1394.0,None


#### **Función de mibian.BS**

Dataframe opciones: Strike, Precio de la opción y la fecha.

Dataframe futuros: underlying price (subyacente) y fecha

 - **underlying_price:** Precio subyacente, es el precio del futuro MINI IBEX al vencimiento más próximo.
 
 - **call price:** Es el precio de la opción, columna 'ANT'

 - **interest rate:** 0

 - **days to expiration:** Diferencia entre la fecha actual y la de vencimiento